# Simple aggregation examples

This notebook illustrates some simple aggregations over genotype data via scikit-allel and dask, using data from 1000 genomes phase 3. Assumes data previously converted to zarr format.

These examples demonstrate some one-step and two-step dask computations.

## Setup

In [1]:
from pathlib import Path
import sys
import functools
import numpy as np
import pandas as pd

In [2]:
import allel
allel.__version__

'1.1.10'

In [3]:
import zarr
zarr.__version__

'2.2.0'

In [4]:
data_path = Path('../data/1000genomes/release/20130502')
zarr_path = data_path / 'zarr'

In [5]:
store = zarr.DirectoryStore(str(zarr_path))
callset = zarr.Group(store=store, read_only=True)
callset

<zarr.hierarchy.Group '/' read-only>

In [6]:
# fix on a chromosome
chrom = '22'

In [7]:
# access genotype data
gtz = callset[chrom]['calldata/GT']
gtz

<zarr.core.Array '/22/calldata/GT' (1103547, 2504, 2) int8 read-only>

In [8]:
gtz.info

Name,/22/calldata/GT
Type,zarr.core.Array
Data type,int8
Shape,"(1103547, 2504, 2)"
Chunk shape,"(65536, 64, 2)"
Order,C
Read-only,True
Compressor,"Blosc(cname='zstd', clevel=1, shuffle=AUTOSHUFFLE, blocksize=0)"
Store type,zarr.storage.DirectoryStore
No. bytes,5526563376 (5.1G)
No. bytes stored,74640281 (71.2M)


In [9]:
# wrap with scikit-allel array class
gt = allel.GenotypeDaskArray(gtz)
gt

<GenotypeDaskArray shape=(1103547, 2504, 2) dtype=int8>

In [10]:
# Download sample metadata.
!cd {data_path} && wget --no-clobber ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/integrated_call_samples_v3.20130502.ALL.panel

File ‘integrated_call_samples_v3.20130502.ALL.panel’ already there; not retrieving.


In [11]:
# Setup sample dataframe
df_samples = (
    pd.read_csv(data_path / 'integrated_call_samples_v3.20130502.ALL.panel', sep='\t')
    [['sample', 'pop', 'super_pop', 'gender']]
)
df_samples.head()

,sample,pop,super_pop,gender
0,HG00096,GBR,EUR,male
1,HG00097,GBR,EUR,female
2,HG00099,GBR,EUR,female
3,HG00100,GBR,EUR,female
4,HG00101,GBR,EUR,male


N.B., here we can assume that the order of the samples in the metadata file is the same as the order of the samples in the VCF (and therefore Zarr) files. 

## Count alleles

The allele count computation simply counts, for each variant, the number of times each allele is observed. I.e., for each row, count how many 0s, 1s, 2s, etc.

Some extra computation can be avoided if we know ahead of time what the largest allele value is within the genotype array. We can obtain this from the number of ALT values.

In [12]:
max_allele = callset[chrom]['variants/ALT'].shape[1]
max_allele

8

This computation will be run via dask. By default, dask will use the multithreaded scheduler. See the dask documentation for how to configure different schedulers.

### All samples

Run the allele count over the whole cohort (i.e., all samples).

In [13]:
%%time
ac = gt.count_alleles(max_allele=max_allele).compute()

CPU times: user 32.3 s, sys: 454 ms, total: 32.8 s
Wall time: 4.44 s


In [14]:
ac

<AlleleCountsArray shape=(1103547, 9) dtype=int64>
5007    1    0    0    0    0    0    0    0
4976   32    0    0    0    0    0    0    0
4970   38    0    0    0    0    0    0    0
...
4969   39    0    0    0    0    0    0    0
5007    1    0    0    0    0    0    0    0
4989   19    0    0    0    0    0    0    0

### Sample subset

Now run the allele count computation over a subset of samples. First select the samples to run, e.g., all Africans.

In [15]:
# Obtain indices for African samples.
loc_african_samples = df_samples[df_samples.super_pop == 'AFR'].index.values
len(loc_african_samples)

661

Run the computation. N.B., this is a two-step computation.

In [27]:
%%time
ac_afr = gt.take(loc_african_samples, axis=1).count_alleles(max_allele=max_allele).compute()

CPU times: user 42.4 s, sys: 260 ms, total: 42.7 s
Wall time: 5.61 s


In [28]:
ac_afr

<AlleleCountsArray shape=(1103547, 9) dtype=int64>
1322    0    0    0    0    0    0    0    0
1291   31    0    0    0    0    0    0    0
1286   36    0    0    0    0    0    0    0
...
1322    0    0    0    0    0    0    0    0
1322    0    0    0    0    0    0    0    0
1322    0    0    0    0    0    0    0    0

## Count genotypes

Another simple aggregation is to count the number of occurrences of a given genotype (e.g., missing, homozygous reference, heterozygous, or homozygous alternate). This aggregation can either be performed over samples (i.e., count genotypes per variant) or over variants (i.e., count genotypes per sample).

### Count per variant - all samples

In [18]:
%%time
hets_per_variant = gt.count_het(axis=1).compute()

CPU times: user 1min 12s, sys: 192 ms, total: 1min 12s
Wall time: 9.31 s


In [19]:
hets_per_variant

array([ 1, 32, 36, ..., 37,  1, 19])

### Count per variant - sample subset

In [20]:
%%time
hets_per_variant_afr = gt.take(loc_african_samples, axis=1).count_het(axis=1).compute()

CPU times: user 41.4 s, sys: 109 ms, total: 41.6 s
Wall time: 5.42 s


In [45]:
hets_per_variant_afr

array([ 0, 31, 34, ...,  0,  0,  0])

### Count per sample - all variants

In [22]:
%%time
hets_per_sample = gt.count_het(axis=0).compute()

CPU times: user 1min 10s, sys: 127 ms, total: 1min 10s
Wall time: 9.09 s


In [23]:
hets_per_sample

array([30154, 34053, 35291, ..., 40246, 40320, 38269])

### Count per sample - variant subset

Illustrate running a computation that first selects a subset of variants.

In [24]:
# locate variants segregating in Africa
loc_african_variants = ac_afr.is_segregating()
np.count_nonzero(loc_african_variants)

573660

In [29]:
%%time
hets_per_sample_afr = gt.compress(loc_african_variants, axis=0).count_het(axis=0).compute()

CPU times: user 43.1 s, sys: 264 ms, total: 43.3 s
Wall time: 5.84 s


In [30]:
hets_per_sample_afr

array([29612, 33338, 34654, ..., 38853, 38568, 36925])